<center><h1>Accidents Corporels de la Route en France</h1></center>
<br>
<center><i>Projet Data Visualization - IGR204</i></center>
<center><i>Télécom ParisTech</i></center>

## General imports

In [1]:
# General
import json
import numpy as np
import pandas as pd
import itertools
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Graph
import matplotlib.pyplot as plt
import matplotlib.offsetbox as offsetbox
import seaborn as sns

## I - Import preprocessed data

In [3]:
# Merge dataframe
df = pd.read_csv('preprocessed_data.csv')

# Show head
df.head()

,Unnamed: 0,Num_Acc,num_veh,vehicule,grav,secu,road,pieton,sexe
0,0,200500000001,A01,Voiture,Blessure,Oui,Départementale,Non,M
1,1,200500000001,B02,Voiture,Blessure,Oui,Départementale,Non,F
2,2,200500000001,B02,Voiture,Indemne,Oui,Départementale,Non,M
3,3,200500000001,B02,Voiture,Indemne,Oui,Départementale,Non,M
4,4,200500000001,B02,Voiture,Indemne,Oui,Départementale,Non,M


## II  - Sankey Diagram

### II - a) Nodes information

In [4]:
# Build dict for vehicule
Vehicule = {'Voiture':{'node':0, 'label':'Voiture', 'color':'#7e4eb0'},
            'Moto':{'node':1 , 'label':'Moto', 'color':'#586fc1'},
            'Vélo':{'node':2 , 'label':'Vélo', 'color':'#7ca3f4'},
            'Camion':{'node':3 , 'label':'Camion', 'color':'#82d4ca'}}

# Build dict for injury
Injury = {'Décès':{'node':4 , 'label':'Décès', 'color':'#ff566f', 'alpha':1},
          'Blessure':{'node':5 , 'label':'Blessure', 'color':'#ffcf35', 'alpha':0.5},
          'Indemne':{'node':6 , 'label':'Indemne', 'color':'#3eff6d', 'alpha':0.1}}
    
# Build dict for security
Security = {'Oui':{'node':7 , 'label':'Oui', 'color':'#08705f','alpha':0.4, 'name':'Sécurité'},
            'Non':{'node':8 , 'label':'Non', 'color':'#b84d2b', 'alpha':1, 'name':'Pas de sécurité'}}

# Build dict for pedestrian
Pedestrian = {'Oui':{'node':9 , 'label':'Oui', 'color':'#801407', 'alpha':0.3, 'name':'Piéton'}}

### II - b) Nodes and Link for each roads

In [5]:
# Build Sankey Dict(node, links) for each road
for road in df['road'].unique().tolist():
    
    links = []
    nodes = []
    
    # Get only accident for type of road
    df_road = df[df['road'] == road]
    
    # Get link between vehicule and injury
    for key_v in Vehicule:

        for key_i in Injury:

            # Link source 
            source = Vehicule[key_v]['node']

            # Link target
            target = Injury[key_i]['node']

            # Link value 
            value = df_road[(df_road['vehicule'] == key_v) & (df_road['grav'] == key_i)].shape[0]

            # Link color 
            link_color = Vehicule[key_v]['color']

            # Link alpha
            link_alpha = Injury[key_i]['alpha']
            
            # Link label 
            link_legend = 'Vehicule: {}<br>Gravité: {}<br>Total: {}'.format(Vehicule[key_v]['label'], Injury[key_i]['label'], value)
            
            # Add to link dict
            links.append({'source':source, 'target':target, 'value':value, 'color':link_color, 'alpha':link_alpha, 'legend':link_legend})
            
            
    # Get link between injury and security
    for key_i in Injury:

        # Security link
        for key_s in Security:

            # Link source 
            source = Injury[key_i]['node']

            # Link target
            target = Security[key_s]['node']

            # Link value 
            value = df_road[(df_road['secu'] == key_s) & (df_road['grav'] == key_i)].shape[0]

            # Link color
            link_color = Injury[key_i]['color']
            
            # Link alpha 
            link_alpha = Security[key_s]['alpha']

            # Link label 
            link_legend = 'Gravité: {}<br>Utilisation de la sécurité: {}<br>Total: {}'.format(Injury[key_i]['label'], Security[key_s]['label'], value)
            
            # Add to link dict
            links.append({'source':source, 'target':target, 'value':value, 'color':link_color, 'alpha':link_alpha, 'legend':link_legend})
            
        # Obstacle link
        if key_i in ['Décès', 'Blessure']:
            for key_p in Pedestrian:

                # Link source 
                source = Injury[key_i]['node']

                # Link target
                target = Pedestrian[key_p]['node']

                # Link value 
                value = df_road[(df_road['secu'] == key_s) & (df_road['grav'] == key_i) & (df_road['pieton'] == key_p)].shape[0]

                # Link color
                link_color = Pedestrian[key_p]['color']
                
                # Link alpha 
                link_alpha = Pedestrian[key_p]['alpha']

                # Link label 
                link_legend = 'Gravité: {}<br>Piéton impliqué: {}<br>Total: {}'.format(Injury[key_i]['label'], Pedestrian[key_p]['label'], value)

                # Add to link dict
                links.append({'source':source, 'target':target, 'value':value, 'color':link_color, 'alpha':link_alpha, 'legend':link_legend})
            
                
    # Create node Label and colors for vehicule
    for key_v in Vehicule:
        name = key_v
        node = Vehicule[key_v]['node']
        color = Vehicule[key_v]['color']
        value = df_road[(df_road['vehicule'] == key_v)].shape[0]
        legend = 'Vehicule: {}<br> Total: {}'.format(name, value)
        nodes.append({"node": node, "name":name, 'color':color, 'legend':legend})

    # Create node label and colors for injury
    for key_i in Injury:
        name = key_i
        node = Injury[key_i]['node']
        color = Injury[key_i]['color']
        value = df_road[(df_road['grav'] == key_i)].shape[0]
        legend = 'Gravité: {}<br> Total: {}'.format(name, value)
        nodes.append({"node": node, "name":name, 'color':color, 'legend':legend})

    # Create node Label and colors for security
    for key_s in Security:
        name = Security[key_s]['name']
        node = Security[key_s]['node']
        color = Security[key_s]['color']
        value = df_road[(df_road['secu'] == key_s)].shape[0]
        legend = 'Sécurité utilisée: {}<br> Total: {}'.format(Security[key_s]['label'], value)
        nodes.append({"node": node, "name":name, 'color':color, 'legend':legend})

    # Create node Label and colors for pedestrian
    for key_p in Pedestrian:
        name = Pedestrian[key_p]['name']
        node = Pedestrian[key_p]['node']
        color = Pedestrian[key_p]['color']
        value = df_road[(df_road['pieton'] == key_p)].shape[0]
        legend = 'Piéton impliqué: {}<br> Total: {}'.format(Pedestrian[key_p]['label'], value)
        nodes.append({"node": node, "name":name, 'color':color, 'legend':legend})

    # Save as JSON
    dict_link_node = {'nodes':nodes, 'links':links}
    with open('New/' + road +'.json', 'w') as fp:
        json.dump(dict_link_node, fp)


### II - c) Nodes and links for all roads

In [6]:

links = []
nodes = []

# Get link between vehicule and injury
for key_v in Vehicule:

    for key_i in Injury:

        # Link source 
        source = Vehicule[key_v]['node']

        # Link target
        target = Injury[key_i]['node']

        # Link value 
        value = df[(df['vehicule'] == key_v) & (df['grav'] == key_i)].shape[0]

        # Link color 
        link_color = Vehicule[key_v]['color']

        # Link alpha
        link_alpha = Injury[key_i]['alpha']

        # Link label 
        link_legend = 'Vehicule: {}<br>Gravité: {}<br>Total: {}'.format(Vehicule[key_v]['label'], Injury[key_i]['label'], value)

        # Add to link dict
        links.append({'source':source, 'target':target, 'value':value, 'color':link_color, 'alpha':link_alpha, 'legend':link_legend})


# Get link between injury and security
for key_i in Injury:

    # Security link
    for key_s in Security:

        # Link source 
        source = Injury[key_i]['node']

        # Link target
        target = Security[key_s]['node']

        # Link value 
        value = df[(df['secu'] == key_s) & (df['grav'] == key_i)].shape[0]

        # Link color
        link_color = Injury[key_i]['color']

        # Link alpha 
        link_alpha = Security[key_s]['alpha']

        # Link label 
        link_legend = 'Gravité: {}<br>Utilisation de la sécurité: {}<br>Total: {}'.format(Injury[key_i]['label'], Security[key_s]['label'], value)

        # Add to link dict
        links.append({'source':source, 'target':target, 'value':value, 'color':link_color, 'alpha':link_alpha, 'legend':link_legend})

    # Obstacle link
    if key_i in ['Décès', 'Blessure']:
        for key_p in Pedestrian:

            # Link source 
            source = Injury[key_i]['node']

            # Link target
            target = Pedestrian[key_p]['node']

            # Link value 
            value = df[(df['secu'] == key_s) & (df['grav'] == key_i) & (df['pieton'] == key_p)].shape[0]

            # Link color
            link_color = Pedestrian[key_p]['color']

            # Link alpha 
            link_alpha = Pedestrian[key_p]['alpha']

            # Link label 
            link_legend = 'Gravité: {}<br>Piéton impliqué: {}<br>Total: {}'.format(Injury[key_i]['label'], Pedestrian[key_p]['label'], value)

            # Add to link dict
            links.append({'source':source, 'target':target, 'value':value, 'color':link_color, 'alpha':link_alpha, 'legend':link_legend})


# Create node Label and colors for vehicule
for key_v in Vehicule:
    name = key_v
    node = Vehicule[key_v]['node']
    color = Vehicule[key_v]['color']
    value = df[(df['vehicule'] == key_v)].shape[0]
    legend = 'Vehicule: {}<br> Total: {}'.format(name, value)
    nodes.append({"node": node, "name":name, 'color':color, 'legend':legend})

# Create node label and colors for injury
for key_i in Injury:
    name = key_i
    node = Injury[key_i]['node']
    color = Injury[key_i]['color']
    value = df[(df['grav'] == key_i)].shape[0]
    legend = 'Gravité: {}<br> Total: {}'.format(name, value)
    nodes.append({"node": node, "name":name, 'color':color, 'legend':legend})

# Create node Label and colors for security
for key_s in Security:
    name = Security[key_s]['name']
    node = Security[key_s]['node']
    color = Security[key_s]['color']
    value = df[(df['secu'] == key_s)].shape[0]
    legend = 'Sécurité utilisée: {}<br> Total: {}'.format(Security[key_s]['label'], value)
    nodes.append({"node": node, "name":name, 'color':color, 'legend':legend})

# Create node Label and colors for pedestrian
for key_p in Pedestrian:
    name = Pedestrian[key_p]['name']
    node = Pedestrian[key_p]['node']
    color = Pedestrian[key_p]['color']
    value = df[(df['pieton'] == key_p)].shape[0]
    legend = 'Piéton impliqué: {}<br> Total: {}'.format(Pedestrian[key_p]['label'], value)
    nodes.append({"node": node, "name":name, 'color':color, 'legend':legend})

# Save as JSON
dict_link_node = {'nodes':nodes, 'links':links}
with open('New/' + 'Complet' +'.json', 'w') as fp:
    json.dump(dict_link_node, fp)
